In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_stata('data/SUF_DEAS2021_1.1_en.dta')
print(df.shape)

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_33429/3336401604.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata('data/SUF_DEAS2021_1.1_en.dta')


(5402, 2117)


In [3]:
useful_variablee = ['fallnum',
'alter_21', 'kc1', 'bmi_21', 'sf36_21', 'anzphy_21', 'lz_21', 'selbstwirk_21', 'kc300', 
'anzkind_21', 'kc3031', 'kc303a1', 'kc303b1', 'kc3021', 'kc3041', 'kc518a9', 
'kc518a11', 'kc518a1', 'kc518a2', 'kc518a3', 'kc518a4', 'kc518a5', 'kc518a6', 'kc518a7', 
'kc518a8', 'kc518a10', 'kc518a11', 'kc518a12', 'kc518a13', 'kc518a14', 'kc518a15', 
'kc518a16', 'kc518a17', 'kc518a18', 'kc518a19', 'kc518a20', 'kc503b', 'kc148_1', 'kd43_1', 
'kd44_1', 'kd45_1', 'kd43_21', 'kd44_21', 'kd45_21', 'kd43_22', 'kd44_22', 'kd45_22',
'bildung4_21', 'isced_21', 'anzphy_21'
]

##### change name of some variables

In [4]:
df = df.rename(columns={'fallnum': 'ID',
                        'alter_21':'age', 
                        'bmi_21':'bmi', 
                        'kc1':'sex', 
                        'bildung4_21': 'level_education',
                        'kc200' : 'marital_status',
                        'kc300':'N_children',
                        'kc518a9': 'OA',
                        'kc802_1':'total_household_income',
                        'kc3041': 'age_first_children', # what age they got pregnant (new variable)
                        
                        'kc518a1' : 'High cholesterol',
                        'kc518a2' : 'Diabetes',
                        'kc518a3' : 'blood_pressure',
                        'kc518a4' : 'Heart_attack',
                        'kc518a5' : 'Cardiac_insufficiency',
                        'kc518a6' : 'Stroke',
                        'kc518a7' : 'Circulatory_disorders_brain',
                        'kc518a8' : 'Circulatory_disorders_legs',
                        'kc518a10' : 'Osteoporosis',
                        'kc518a11' : 'Inflammatory_joint',
                        'kc518a12' : 'Chronic_pulmonary',
                        'kc518a13' : 'Cancer',
                        'kc518a14' : 'Stomach_ulcer',
                        'kc518a15' : 'Incontinence',
                        'kc518a16' : 'Mental_illness',
                        'kc518a17' : 'Parkinson_disease',
                        'kc518a18' : 'Glaucoma',
                        'kc518a19' : 'Other_chronic_disease',
                        'kc518a20' : 'other_illness',
                        
                        'lz_21': 'life_satisfication',
                        'kc503b': 'General_pain', # Constant or recurring pain in the last four weeks
                        'isced_21':'level_education_ISCED',                       
                        'kc808': 'lowest_income',     
                        'anzphy_21': 'N_physical_diseases', 

                        })

##### Dataframe with desire variables:

In [5]:
variables = ['ID', 'age', 'sex', 'bmi', 'N_children', 'OA', 
              'age_first_children', 'level_education',  
             
             'High cholesterol', 'Diabetes', 'blood_pressure', 'Heart_attack',
             'Cardiac_insufficiency', 'Stroke', 'Circulatory_disorders_brain',
             'Circulatory_disorders_legs', 'Osteoporosis', 'Inflammatory_joint',
             'Chronic_pulmonary', 'Cancer', 'Stomach_ulcer', 'Incontinence',
             'Mental_illness', 'Parkinson_disease', 'Glaucoma',
             'Other_chronic_disease', 'other_illness']
df = df[variables]
print(df.shape)

(5402, 27)


In [6]:
df.isna().sum()

ID                               0
age                              0
sex                              0
bmi                             80
N_children                       3
OA                               0
age_first_children             671
level_education                  0
High cholesterol                 0
Diabetes                         0
blood_pressure                   0
Heart_attack                     0
Cardiac_insufficiency            0
Stroke                           0
Circulatory_disorders_brain      0
Circulatory_disorders_legs       0
Osteoporosis                     0
Inflammatory_joint               0
Chronic_pulmonary                0
Cancer                           0
Stomach_ulcer                    0
Incontinence                     0
Mental_illness                   0
Parkinson_disease                0
Glaucoma                         0
Other_chronic_disease            0
other_illness                    0
dtype: int64

In [7]:
# unique values
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print("\n" + "-"*40 + "\n") 

Unique values in column 'ID':
[1000007 1000018 1000020 ... 4005998 4005999 4006002]

----------------------------------------

Unique values in column 'age':
[71, 83, 72, 80, '97-100', ..., 51, 50, 47, 48, 46]
Length: 51
Categories (51, object): [46 < 47 < 48 < 49 ... 93 < 94 < '95-96' < '97-100']

----------------------------------------

Unique values in column 'sex':
['female', 'male']
Categories (2, object): ['male' < 'female']

----------------------------------------

Unique values in column 'bmi':
[16.852522, 25.249337, 30.119376, 37.109375, 22.959088, ..., 29.142146, 33.984375, 30.026595, 20.370370, 17.592593]
Length: 1640
Categories (1639, float64): [15.203813 < 15.234375 < 15.780836 < 15.886966 ... 54.815315 < 56.689342 < 56.923596 < 58.792443]

----------------------------------------

Unique values in column 'N_children':
['no children', 1.0, 2.0, 3.0, 5.0, 4.0, 6.0, '8+', NaN, 7.0]
Categories (9, object): ['no children' < 1.0 < 2.0 < 3.0 ... 5.0 < 6.0 < 7.0 < '8+']

------

##### Creating new variable: Number of kids (N_kids)

In [8]:
def create_N_kids(df):
    """
    Function to create the 'N_kids' variable based on the 'N_children' column.
    
    Rules:
    - 'no children' -> 0
    - 1, 2, 3 -> 1, 2, 3 (same as value)
    - >3 or '8+' -> 4
    - NaN -> NaN
    """
    def map_N_kids(value):
        if value == 'no children':
            return 0
        elif value in [1.0, 2.0, 3.0]:
            return value
        elif value in [4.0, 5.0, 6.0, 7.0, '8+']:  #'
            return 4.0
        else:
            return np.nan  # Handle unexpected values with NaN


    df['N_kids'] = df['N_children'].apply(map_N_kids)
    
    return df


In [9]:
print("Number of kids/before", df.N_children.value_counts())
print("number of miss value:", df.N_children.isna().sum())
df = create_N_kids(df)
print("-----------------------")
print("number of miss value:", df.N_kids.isna().sum())
print("Number of kids/after", df.N_kids.value_counts())

Number of kids/before N_children
2.0            2308
1.0            1221
3.0             844
no children     649
4.0             265
5.0              72
6.0              23
7.0              11
8+                6
Name: count, dtype: int64
number of miss value: 3
-----------------------
number of miss value: 3
Number of kids/after N_kids
2.0    2308
1.0    1221
3.0     844
0.0     649
4.0     377
Name: count, dtype: int64


In [10]:
df = create_N_kids(df)
print("-----------------------")
print("number of miss value:", df.N_kids.isna().sum())
print("Number of kids/befor droping nan value", df.N_kids.value_counts())

-----------------------
number of miss value: 3
Number of kids/befor droping nan value N_kids
2.0    2308
1.0    1221
3.0     844
0.0     649
4.0     377
Name: count, dtype: int64


In [11]:
print("Number of kids/before", df.N_kids.value_counts())
print("number of miss value:", df.N_kids.isna().sum())

Number of kids/before N_kids
2.0    2308
1.0    1221
3.0     844
0.0     649
4.0     377
Name: count, dtype: int64
number of miss value: 3


##### Creating new variables: Has kid? (has_kid)

In [12]:
# has kid?
def map_has_kid(value):
    """ Map 'N_children' to 'has_kid' according to specified rules """
    if pd.isna(value):
        return np.nan  # Return NaN if value is NaN
    elif value == 0.0:
        return 0
    elif value in [1.0, 2.0, 3.0, 4.0]:
        return 1
    else:
        return np.nan

In [13]:
print("Number of kids/before", df.N_kids.value_counts())
print("number of miss value:", df.N_kids.isna().sum())
df['has_kid'] = df['N_kids'].apply(map_has_kid)
print("-----------------------")
print("number of miss value:", df.has_kid.isna().sum())
print("Number of kids/after", df.has_kid.value_counts())

Number of kids/before N_kids
2.0    2308
1.0    1221
3.0     844
0.0     649
4.0     377
Name: count, dtype: int64
number of miss value: 3
-----------------------
number of miss value: 3
Number of kids/after has_kid
1.0    4750
0.0     649
Name: count, dtype: int64


##### Maping value of disease variables:

In [14]:
def map_cholesterol(df, column_name='High cholesterol'):
    """
    Maps the 'cholesterol' column to numerical values 
    """
    cholesterol_mapping = {
        'No': 0, 
        'Yes': 1, 
    }
    
    df[column_name] = df[column_name].map(cholesterol_mapping)
    return df

def map_disease(value):
    """
    Map disease presence to 1 and absence to 0, or return NaN for unknown values.
    """
    if value == 'mentioned':
        return 1
    elif value == 'not mentioned':
        return 0
    else:
        return np.nan

# Function to apply map_disease to specified columns
def apply_disease_mapping(df):
    disease_columns = [
        'Diabetes', 'blood_pressure', 'Heart_attack',
        'Cardiac_insufficiency', 'Stroke', 'Circulatory_disorders_brain',
        'Circulatory_disorders_legs', 'Osteoporosis', 'Inflammatory_joint',
        'Chronic_pulmonary', 'Cancer', 'Stomach_ulcer', 'Incontinence',
        'Mental_illness', 'Parkinson_disease', 'Glaucoma',
        'Other_chronic_disease', 'other_illness'
    ]
    
 
    for col in disease_columns:
        df[col] = df[col].apply(map_disease)
    
    return df

In [15]:
disease_columns = [
        'Diabetes', 'blood_pressure', 'Heart_attack',
        'Cardiac_insufficiency', 'Stroke', 'Circulatory_disorders_brain',
        'Circulatory_disorders_legs', 'Osteoporosis', 'Inflammatory_joint',
        'Chronic_pulmonary', 'Cancer', 'Stomach_ulcer', 'Incontinence',
        'Mental_illness', 'Parkinson_disease', 'Glaucoma',
        'Other_chronic_disease', 'other_illness', 'High cholesterol'
    ]
for col in disease_columns:
    print(f"{col}: {df[col].isna().sum()}")
    


Diabetes: 0
blood_pressure: 0
Heart_attack: 0
Cardiac_insufficiency: 0
Stroke: 0
Circulatory_disorders_brain: 0
Circulatory_disorders_legs: 0
Osteoporosis: 0
Inflammatory_joint: 0
Chronic_pulmonary: 0
Cancer: 0
Stomach_ulcer: 0
Incontinence: 0
Mental_illness: 0
Parkinson_disease: 0
Glaucoma: 0
Other_chronic_disease: 0
other_illness: 0
High cholesterol: 0


In [16]:
df = apply_disease_mapping(df)
df = map_cholesterol(df)

In [17]:
for col in disease_columns:
    print(f"{col}: {df[col].isna().sum()}")

Diabetes: 0
blood_pressure: 0
Heart_attack: 0
Cardiac_insufficiency: 0
Stroke: 0
Circulatory_disorders_brain: 0
Circulatory_disorders_legs: 0
Osteoporosis: 0
Inflammatory_joint: 0
Chronic_pulmonary: 0
Cancer: 0
Stomach_ulcer: 0
Incontinence: 0
Mental_illness: 0
Parkinson_disease: 0
Glaucoma: 0
Other_chronic_disease: 0
other_illness: 0
High cholesterol: 0


In [18]:
for col in disease_columns:
    print(f"{col}: {df[col].value_counts()}")

Diabetes: Diabetes
0    4679
1     723
Name: count, dtype: int64
blood_pressure: blood_pressure
0    2763
1    2639
Name: count, dtype: int64
Heart_attack: Heart_attack
0    5130
1     272
Name: count, dtype: int64
Cardiac_insufficiency: Cardiac_insufficiency
0    4671
1     731
Name: count, dtype: int64
Stroke: Stroke
0    5165
1     237
Name: count, dtype: int64
Circulatory_disorders_brain: Circulatory_disorders_brain
0    5287
1     115
Name: count, dtype: int64
Circulatory_disorders_legs: Circulatory_disorders_legs
0    4830
1     572
Name: count, dtype: int64
Osteoporosis: Osteoporosis
0    4938
1     464
Name: count, dtype: int64
Inflammatory_joint: Inflammatory_joint
0    4581
1     821
Name: count, dtype: int64
Chronic_pulmonary: Chronic_pulmonary
0    4932
1     470
Name: count, dtype: int64
Cancer: Cancer
0    4831
1     571
Name: count, dtype: int64
Stomach_ulcer: Stomach_ulcer
0    5238
1     164
Name: count, dtype: int64
Incontinence: Incontinence
0    5147
1     255
Name:

##### Create new variable: number of diseases (N_disease)

In [19]:
# Create new variable: disease_columns which  is number of disease that made from diseases
N_disease_columns = [
    'Diabetes', 'blood_pressure', 'Heart_attack',
        'Cardiac_insufficiency', 'Stroke', 'Circulatory_disorders_brain',
        'Circulatory_disorders_legs', 'Osteoporosis', 'Inflammatory_joint',
        'Chronic_pulmonary', 'Cancer', 'Stomach_ulcer', 'Incontinence',
        'Mental_illness', 'Parkinson_disease', 'Glaucoma',
        'Other_chronic_disease', 'other_illness', 'High cholesterol'
]

df[N_disease_columns] = df[N_disease_columns].apply(pd.to_numeric, errors='coerce')


df['N_disease'] = df[N_disease_columns].sum(axis=1)

In [20]:
print("number of miss value:", df.N_disease.isna().sum())
print("value count/after", df.N_disease.value_counts())

number of miss value: 0
value count/after N_disease
1     2039
2     1159
3      821
4      517
0      294
5      288
6      154
7       79
8       31
9       12
10       6
11       1
12       1
Name: count, dtype: int64


##### Map value of variables, (age, sex, OA, Education)

In [21]:
# age:
def map_age(value):
    """ Convert age range to an integer average, or keep the value as is."""
    if isinstance(value, str) and '-' in value:
        parts = value.split('-')
        try:
            start = int(parts[0])
            end = int(parts[1])
            return int((start + end) / 2)
        except:
            return np.nan  
    else:
        try:
            return int(value)  
        except:
            return np.nan  
# sex:
def map_sex(value):
    """ Map 'female' to 1 and 'male' to 0 """
    value = value.strip().lower()
    if value == 'female':
        return 1
    elif value == 'male':
        return 0
    else:
        return np.nan
# OA:
def map_OA(value):
    """ Map 'no OA' to 0 and 'has OA' to 1 """
    value = value.strip().lower()  #
    if value == 'not mentioned':
        return 0
    elif value == 'mentioned':
        return 1
    else:
        return np.nan 

# Education
def map_level_education(df, column_name='level_education'):
    """
    Maps the 'level_education' column to numerical values 
    """
    education_mapping = {
        'low': 0,
        'medium': 1, 
        'sophisticated': 2, 
        'high': 3,
    }
    
    df[column_name] = df[column_name].map(education_mapping)
    return df

In [22]:
print("----Age----")
print("number of miss value:", df.age.isna().sum())
print("value count/after", df.age.value_counts())
print("----Sex----")
print("number of miss value:", df.sex.isna().sum())
print("value count/after", df.sex.value_counts())
print("----OA----")
print("number of miss value:", df.OA.isna().sum())
print("value count/after", df.OA.value_counts())
print("----Education----")
print("number of miss value:", df.level_education.isna().sum())
print("value count/after", df.level_education.value_counts())

----Age----
number of miss value: 0
value count/after age
65        199
66        186
63        180
64        178
70        174
77        173
71        170
67        170
68        167
72        166
69        166
76        164
62        163
61        162
60        160
80        155
78        152
73        146
59        145
79        145
58        143
81        142
82        135
74        128
57        126
55        125
56        124
83        120
75        111
54        110
84         90
53         87
52         81
85         78
86         64
51         60
87         48
49         46
50         36
48         36
47         34
89         33
88         28
90         26
46         23
91         14
92         11
93         10
97-100      5
94          4
95-96       3
Name: count, dtype: int64
----Sex----
number of miss value: 0
value count/after sex
female    2746
male      2656
Name: count, dtype: int64
----OA----
number of miss value: 0
value count/after OA
not mentioned    2909
mentioned 

In [23]:
# Apply age 
df['age'] = df['age'].apply(map_age)
# Apply sex  
df['sex'] = df['sex'].apply(map_sex)
# Apply OA  
df['OA'] = df['OA'].apply(map_OA)
# Apply Education
df = map_level_education(df)

In [24]:
print("----Age----")
print("number of miss value:", df.age.isna().sum())
print("value count/after", df.age.value_counts())
print("----Sex----")
print("number of miss value:", df.sex.isna().sum())
print("value count/after", df.sex.value_counts())
print("----OA----")
print("number of miss value:", df.OA.isna().sum())
print("value count/after", df.OA.value_counts())
print("----Education----")
print("number of miss value:", df.level_education.isna().sum())
print("value count/after", df.level_education.value_counts())

----Age----
number of miss value: 0
value count/after age
65    199
66    186
63    180
64    178
70    174
77    173
71    170
67    170
68    167
72    166
69    166
76    164
62    163
61    162
60    160
80    155
78    152
73    146
59    145
79    145
58    143
81    142
82    135
74    128
57    126
55    125
56    124
83    120
75    111
54    110
84     90
53     87
52     81
85     78
86     64
51     60
87     48
49     46
50     36
48     36
47     34
89     33
88     28
90     26
46     23
91     14
92     11
93     10
98      5
94      4
95      3
Name: count, dtype: int64
----Sex----
number of miss value: 0
value count/after sex
1    2746
0    2656
Name: count, dtype: int64
----OA----
number of miss value: 0
value count/after OA
0    2909
1    2493
Name: count, dtype: int64
----Education----
number of miss value: 0
value count/after level_education
1    2545
3    1821
2     804
0     232
Name: count, dtype: int64


##### Create new variable: age of having first kid (age_having_kids)

In [ ]:
def map_age_first_children(value):
    if isinstance(value, str):
        # Check if the value contains a range
        if '-' in value:
            # Extract the first and last year from the range
            years = value.split('-')
            try:
                # Take the average of the range
                start_year = int(years[0].split()[-1])  # Extract the year before the range
                end_year = int(years[1])
                return int((start_year + end_year) / 2)
            except ValueError:
                return np.nan  # Handle invalid values
        else:
            # Extract the first year if no range is provided
            try:
                return int(value.split('.')[0])  # Take the first year in the string
            except ValueError:
                return np.nan
    else:
        # If the value is already numeric (e.g., 1949.0), return it as an integer
        return int(value) if pd.notnull(value) else np.nan

In [26]:
print("number of miss value:", df.age_first_children.isna().sum())
print("value count/after", df.age_first_children.value_counts())

number of miss value: 671
value count/after age_first_children
1980.0             172
1982.0             151
1977.0             148
1981.0             143
1969.0             140
                  ... 
1950.0               6
2011. 2010-2011      5
2012.0               4
1949.0               3
2014.0               3
Name: count, Length: 66, dtype: int64


In [27]:
df['age_first_children'] = df['age_first_children'].apply(map_age_first_children)

In [28]:
interview_year = 2021

df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['age_first_children'] = pd.to_numeric(df['age_first_children'], errors='coerce')

# Calculate the age when they had their first child
df['age_having_kids'] = interview_year - df['age_first_children']

df.age_having_kids

0        NaN
1       60.0
2       51.0
3       56.0
4        NaN
        ... 
5397    18.0
5398    55.0
5399    41.0
5400    51.0
5401    23.0
Name: age_having_kids, Length: 5402, dtype: float64

In [29]:
print("number of miss value:", df.age_having_kids.isna().sum())
print("value count/after", df.age_having_kids.value_counts())

number of miss value: 671
value count/after age_having_kids
41.0    172
39.0    151
44.0    148
40.0    143
52.0    140
       ... 
71.0      6
11.0      5
9.0       4
72.0      3
7.0       3
Name: count, Length: 66, dtype: int64


##### Drop NaN value and save dataframe:

In [30]:
columns_to_drop = [
    'High cholesterol', 'Diabetes', 'blood_pressure', 'Heart_attack',
    'Cardiac_insufficiency', 'Stroke', 'Circulatory_disorders_brain',
    'Circulatory_disorders_legs', 'Osteoporosis', 'Inflammatory_joint',
    'Chronic_pulmonary', 'Cancer', 'Stomach_ulcer', 'Incontinence',
    'Mental_illness', 'Parkinson_disease', 'Glaucoma',
    'Other_chronic_disease', 'other_illness',
    
    'age_first_children', 'N_children', 'age_having_kids'
]
df_new = df.drop(columns=columns_to_drop)

In [31]:
print("shape if df: ", df.shape)
print("Shape of df_new:", df_new.shape)

shape if df:  (5402, 31)
Shape of df_new: (5402, 9)


In [33]:
# unique values
for column in df_new.columns:
    unique_values = df_new[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print("\n" + "-"*40 + "\n") 

Unique values in column 'ID':
[1000007 1000018 1000020 ... 4005998 4005999 4006002]

----------------------------------------

Unique values in column 'age':
[71 83 72 80 98 95 79 87 82 91 93 78 76 86 85 94 84 81 90 77 74 70 89 67
 88 68 75 73 69 66 64 65 58 59 60 62 63 61 92 56 53 55 54 57 52 49 51 50
 47 48 46]

----------------------------------------

Unique values in column 'sex':
[1, 0]
Categories (2, int64): [0 < 1]

----------------------------------------

Unique values in column 'bmi':
[16.852522, 25.249337, 30.119376, 37.109375, 22.959088, ..., 29.142146, 33.984375, 30.026595, 20.370370, 17.592593]
Length: 1640
Categories (1639, float64): [15.203813 < 15.234375 < 15.780836 < 15.886966 ... 54.815315 < 56.689342 < 56.923596 < 58.792443]

----------------------------------------

Unique values in column 'OA':
[0, 1]
Categories (2, int64): [0 < 1]

----------------------------------------

Unique values in column 'level_education':
[3, 2, 1, 0]
Categories (4, int64): [0 < 1 < 2 

In [35]:
df_new.isna().sum()

ID                  0
age                 0
sex                 0
bmi                80
OA                  0
level_education     0
N_kids              3
has_kid             3
N_disease           0
dtype: int64

In [36]:
df_clean = df_new.dropna()

In [37]:
print("shape if df: ", df_new.shape)
print("Shape of df_new:", df_clean.shape)

shape if df:  (5402, 9)
Shape of df_new: (5319, 9)


In [38]:
df_clean

,ID,age,sex,bmi,OA,level_education,N_kids,has_kid,N_disease
0,1000007,71,1,16.852522,0,3,0.0,0.0,1
1,1000018,83,0,25.249337,0,2,1.0,1.0,1
2,1000020,72,1,30.119376,0,1,2.0,1.0,3
3,1000024,80,1,37.109375,0,1,3.0,1.0,4
4,1000319,98,1,22.959088,0,1,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...
5397,4005996,47,0,25.507493,0,1,2.0,1.0,1
5398,4005997,76,1,25.402816,1,0,2.0,1.0,3
5399,4005998,71,0,17.592593,1,3,2.0,1.0,3
5400,4005999,78,0,26.827421,1,3,4.0,1.0,3


In [39]:
def reorder_columns(df):
    """
    Function to reorder columns in a dataframe according to a specified order.
    """
    # Desired order of columns
    column_order = ['ID', 'age', 'sex', 'OA', 'bmi', 'level_education', 'N_kids', 'has_kid', 'N_disease']
    
    # Reorder the dataframe based on the specified column order
    df = df[column_order]
    
    return df

In [ ]:
df2021 = reorder_columns(df_clean)

In [41]:
df2021.to_csv('data/confounder2021.csv')
df = pd.read_csv('data/confounder2021.csv').drop(columns=['Unnamed: 0'])
df.shape

(5319, 9)

In [43]:
# 1. Men without kid vs. Women without kid
men_without_kid = df[(df['sex'] == 0) & (df['has_kid'] == 0)]
women_without_kid = df[(df['sex'] == 1) & (df['has_kid'] == 0)]

print(f"Number of men without kids: {len(men_without_kid)}")
print(f"Number of women without kids: {len(women_without_kid)}")

men_without_kid_with_oa = men_without_kid[men_without_kid['OA'] == 1]
print(f"Number of men without kids who have OA: {len(men_without_kid_with_oa)}")

women_without_kid_with_oa = women_without_kid[women_without_kid['OA'] == 1]
print(f"Number of women without kids who have OA: {len(women_without_kid_with_oa)}")


Number of men without kids: 313
Number of women without kids: 319
Number of men without kids who have OA: 116
Number of women without kids who have OA: 140
